In [ ]:
! mkdir -p audios/aishell2

import shutil
import subprocess
from IPython.display import display, Audio
import re

def aishell2_create_file(
    text_file: str,
    id_texts: list,
    texts: list,
    infer_dir="audios/aishell2",
    copy=False,
):
    with open(text_file, "w") as file:
        used = {}
        for n, line in enumerate(id_texts):
            id = line[:11]
            id1 = line[1:6]
            id2 = line[6:11]
            text_prompt = line[12:].strip()
            
            if id in used:
                used[id] += 1
                suffix = f"infer{used[id]}"
            else:
                used[id] = 0
                suffix = "infer"
            
            audio_source = f"download/aishell2/AISHELL-2/iOS/test/wav/{id1}/{id}.wav"
            audio_prompt = f"{infer_dir}/{id}_prompt.wav"
            shutil.copyfile(audio_source, audio_prompt)
            text = texts[n % len(texts)]
            audio_file = f"{infer_dir}/{id}_{suffix}.wav"
            file.write(f"{text_prompt}\t{audio_prompt}\t{text}\t{audio_file}\n")
    print(open(text_file).read())
    if copy:
        shutil.copyfile(text_file,f'../../../{text_file}')
    return text_file

def create_file(
    text_file:str,
    audio_dir:str,
    id_texts:list,
    infer_texts:list,
    copy=False,
):
    with open(text_file, "w") as file:
        used = {}
        for i, line in enumerate(id_texts):
            splitted = re.split(r"\s+",line)
            filename = splitted[0]
            text_prompt = splitted[1]
            if filename in used:
                used[filename] += 1
                suffix = f"infer{used[filename]}"
            else:
                used[filename] = 0
                suffix = "infer"
            
            audio_prompt = f"{audio_dir}/{filename}.wav"
            text = infer_texts[i % len(infer_texts)]
            infer_audio = f"{audio_dir}/{filename}_{suffix}.wav"
            file.write(f"{text_prompt}\t{audio_prompt}\t{text}\t{infer_audio}\n")
    print(open(text_file).read())
    if copy:
        shutil.copyfile(text_file, f"../../../{text_file}")
    return text_file

def inference(text_file: str, gpu_id=0):
    subprocess.run([   
        "bash",
        "aishell2_run.sh",
        "--gpus",
        str(gpu_id),
        "--text",
        text_file,
        ])

def show_audios(text_file):    
    with open(text_file) as file:
        for n, line in enumerate(file.readlines()):
            print(f"---------------------------{n}--------------------------------------")
            text_prompt, audio_prompt, text, audio_file = line.strip().split("\t")
            display(text_prompt, Audio(audio_prompt), text, Audio(audio_file))

import os

def copy_file(source_file, destination_folder):
    # 检查源文件是否存在
    if not os.path.isfile(source_file):
        print(f"源文件 '{source_file}' 不存在。")
        return

    # 检查目标文件夹是否存在，如果不存在则创建
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # 获取源文件的文件名
    file_name = os.path.basename(source_file)

    # 构建目标文件的完整路径
    destination_file = os.path.join(destination_folder, file_name)

    try:
        # 复制文件
        shutil.copy2(source_file, destination_file)
        print(f"成功将文件 '{source_file}' 复制到 '{destination_file}'。")
    except Exception as e:
        print(f"复制文件时发生错误：{str(e)}")

## AiShell2 测试集上的效果

In [ ]:
import shutil

# 只能从这个id里面选，其他的不行：16，17，19，22，23，25

# 只能用中文逗号，其他的符号不行

id_texts = """
IT0016W0441	你真的这么做吗
IT0022W0457	他们在靠近
IT0023W0090	那群野蛮人已经走了
IT0025W0410	让你吃尽苦头
"""
id_texts = """
IT0017W0333	现在我们的社会地位是平等的
IT0017W0333	现在我们的社会地位是平等的
IT0017W0333	现在我们的社会地位是平等的
IT0017W0333	现在我们的社会地位是平等的
IT0017W0333	现在我们的社会地位是平等的
"""

# IT0016W0450	占国内生产总值约百分之五
# IT0017W0194	你给主考留下了很好的印象
# IT0019W0351	有条叫莱拉的德国牧羊犬
# IT0022W0225	又一道闪电照亮了山洞
# IT0023W0404	就从你房间偷走他们家钥匙
# IT0025W0324	医生被叫去看急症病人

texts = [
    "今天天气晴朗，阳光明媚",
    "我喜欢吃巧克力冰淇淋",
    "请记住带上你的雨伞",
    "那只小猫咪真可爱",
    "明天是个新的开始",
    "让我们一起跳舞吧",
    "我们去公园散步吧",
    "她唱歌声音非常动听",
    "快乐的笑声在空中回荡",
    "愿你有美好的一天",
]

texts = ["我最近在学习一门新的编程语言"]

id_texts = id_texts.strip("\n").splitlines()

text_file = aishell2_create_file("aishell2.txt", id_texts, texts, copy=True)

inference(text_file, gpu_id=4)

show_audios(text_file)

## AIShell2 nice demo

In [ ]:
id_texts = """
IT0016W0441	你真的这么做吗
IT0022W0457	他们在靠近
IT0023W0090	那群野蛮人已经走了
IT0025W0410	让你吃尽苦头
""".strip().splitlines()

texts = [
    "健康的身体是实现梦想和幸福生活的基石",
]

text_file = aishell2_create_file(
    "aishell2_nice.txt",
    infer_dir="audios/aishell2_nice",
    id_texts=id_texts,
    texts=texts,
    copy=True,
)

# inference(text_file,6)

# show_audios(text_file)

## 用自己的声音测试效果

In [ ]:
id_texts = [
    "ldy 昨天我去了一个很美的地方",
    "ldy 昨天我去了一个很美的地方",
    "ldy 昨天我去了一个很美的地方",
    "ldy1 工作完成了",
    # "ytq 昨天我去了一个很美的地方",
    # "ytq1 终于赶上末班车了"
]

texts = ["今天天气真好，适合出去散步", "我们可以一起去看电影或者喝杯咖啡"]

text_file = create_file(
    "test.txt",
    audio_dir="audios/test",
    id_texts=id_texts,
    infer_texts=texts,
)

inference(text_file)

show_audios(text_file)

# others

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

filename = "audios/aishell2/IT0016W0441.wav"
waveform, sr = librosa.load(filename)

plt.figure(figsize=(12, 4))
# librosa.display.waveplot(waveform, sr=sr)
plt.plot(waveform, color="#007acc")
plt.title("Waveform")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
import subprocess


# 运行系统命令并返回输出
def run_command(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    output = result.stdout.strip()
    return output


# 获取服务器商品信息
def get_server_product_info():
    # 处理器信息
    processor_info = run_command("cat /proc/cpuinfo | grep 'model name' | uniq")
    print("Processor Information:")
    print(processor_info)

    # 内存信息
    memory_info = run_command("cat /proc/meminfo | grep 'MemTotal'")
    print("Memory Information:")
    print(memory_info)


# 获取并输出服务器商品信息
get_server_product_info()